<a href="https://colab.research.google.com/github/vicvarar/TFMG5/blob/main/VikLabTFM5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [73]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from google.colab import drive
drive.mount('drive')


Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).


In [74]:
comprasdf=pd.read_csv('https://raw.githubusercontent.com/megatr0ll/TFM/main/steam_data/game_purchase.csv',
            sep=';', names=['User_ID', 'Game_ID', 'Purchase'])

In [75]:
horasdf=pd.read_csv('https://raw.githubusercontent.com/megatr0ll/TFM/main/steam_data/game_play.csv',
            sep=';', names=['User_ID', 'Game_ID', 'Hours'])

In [76]:
itemdf=pd.read_csv('https://raw.githubusercontent.com/megatr0ll/TFM/main/steam_data/item_info.csv',
            sep=';', names=['Game_ID', 'Game Name'])

In [77]:
userdf=pd.read_csv('https://raw.githubusercontent.com/megatr0ll/TFM/main/steam_data/user_info.csv',
            sep=';', names=['New_ID', 'Real_ID'])

In [78]:
userdf.head()


,New_ID,Real_ID
0,New_ID,Real_ID
1,1,151603712
2,2,187131847
3,3,59945701
4,4,53875128


In [79]:
horasdf.head()

,User_ID,Game_ID,Hours
0,User_ID,Game_ID,Hours
1,1,1,273.0
2,1,2,87.0
3,1,3,14.9
4,1,4,12.1


In [80]:
comprasdf.head()

,User_ID,Game_ID,Purchase
0,User_ID,Game_ID,Purchase
1,1,1,1.0
2,1,2,1.0
3,1,3,1.0
4,1,4,1.0


In [81]:
itemdf.head()

,Game_ID,Game Name
0,Game_ID,Game Name
1,1,The Elder Scrolls V Skyrim
2,2,Fallout 4
3,3,Spore
4,4,Fallout New Vegas


In [82]:
# standard library imports
import csv
import datetime as dt
import json
import os
import statistics
import time

# third-party imports
import numpy as np
import pandas as pd
import requests

# customisations - ensure tables sh

In [83]:
def get_request(url, parameters=None):
    """Return json-formatted response of a get request using optional parameters.
    
    Parameters
    ----------
    url : string
    parameters : {'parameter': 'value'}
        parameters to pass as part of get request
    
    Returns
    -------
    json_data
        json-formatted response (dict-like)
    """
    try:
        response = requests.get(url=url, params=parameters)
    except SSLError as s:
        print('SSL Error:', s)
        
        for i in range(5, 0, -1):
            print('\rWaiting... ({})'.format(i), end='')
            time.sleep(1)
        print('\rRetrying.' + ' '*10)
        
        # recusively try again
        return get_request(url, parameters)
    
    if response:
        return response.json()
    else:
        # response is none usually means too many requests. Wait and try again 
        print('No response, waiting 10 seconds...')
        time.sleep(10)
        print('Retrying.')
        return get_request(url, parameters)

In [84]:
url = "https://steamspy.com/api.php"
parameters = {"request": "all"}

# request 'all' from steam spy and parse into dataframe
json_data = get_request(url, parameters=parameters)
steam_spy_all = pd.DataFrame.from_dict(json_data, orient='index')

# generate sorted app_list from steamspy data
app_list = steam_spy_all[['appid', 'name']].sort_values('appid').reset_index(drop=True)

# export disabled to keep consistency across download sessions
app_list.to_csv('app_list.csv', index=False)
!cp app_list.csv "drive/My Drive/"

# read from stored csv
app_list = pd.read_csv('app_list.csv')

# display first few rows
app_list.head()

,appid,name
0,10,Counter-Strike
1,20,Team Fortress Classic
2,30,Day of Defeat
3,40,Deathmatch Classic
4,50,Half-Life: Opposing Force


In [85]:
def get_app_data(start, stop, parser, pause):
    """Return list of app data generated from parser.
    
    parser : function to handle request
    """
    app_data = []
    
    # iterate through each row of app_list, confined by start and stop
    for index, row in app_list[start:stop].iterrows():
        print('Current index: {}'.format(index), end='\r')
        
        appid = row['appid']
        name = row['name']

        # retrive app data for a row, handled by supplied parser, and append to list
        data = parser(appid, name)
        app_data.append(data)

        time.sleep(pause) # prevent overloading api with requests
    
    return app_data


def process_batches(parser, app_list, download_path, data_filename, index_filename,
                    columns, begin=0, end=-1, batchsize=100, pause=1):
    """Process app data in batches, writing directly to file.
    
    parser : custom function to format request
    app_list : dataframe of appid and name
    download_path : path to store data
    data_filename : filename to save app data
    index_filename : filename to store highest index written
    columns : column names for file
    
    Keyword arguments:
    
    begin : starting index (get from index_filename, default 0)
    end : index to finish (defaults to end of app_list)
    batchsize : number of apps to write in each batch (default 100)
    pause : time to wait after each api request (defualt 1)
    
    returns: none
    """
    print('Starting at index {}:\n'.format(begin))
    
    # by default, process all apps in app_list
    if end == -1:
        end = len(app_list) + 1
    
    # generate array of batch begin and end points
    batches = np.arange(begin, end, batchsize)
    batches = np.append(batches, end)
    
    apps_written = 0
    batch_times = []
    
    for i in range(len(batches) - 1):
        start_time = time.time()
        
        start = batches[i]
        stop = batches[i+1]
        
        app_data = get_app_data(start, stop, parser, pause)
      
        rel_path = os.path.join(download_path, data_filename)
        rel_path = data_filename
        # writing app data to file
        with open(rel_path, 'a', newline='', encoding='utf-8') as f:
            writer = csv.DictWriter(f, fieldnames=columns, extrasaction='ignore')
            
            for j in range(3,0,-1):
                print("\rAbout to write data, don't stop script! ({})".format(j), end='')
                time.sleep(0.5)
            
            writer.writerows(app_data)
            print('\rExported lines {}-{} to {}.'.format(start, stop-1, data_filename), end=' ')
            
        apps_written += len(app_data)
        
        idx_path = os.path.join(download_path, index_filename)
        
        # writing last index to file
        with open(idx_path, 'w') as f:
            index = stop
            print(index, file=f)
            
        # logging time taken
        end_time = time.time()
        time_taken = end_time - start_time
        
        batch_times.append(time_taken)
        mean_time = statistics.mean(batch_times)
        
        est_remaining = (len(batches) - i - 2) * mean_time
        
        remaining_td = dt.timedelta(seconds=round(est_remaining))
        time_td = dt.timedelta(seconds=round(time_taken))
        mean_td = dt.timedelta(seconds=round(mean_time))
        
        print('Batch {} time: {} (avg: {}, remaining: {})'.format(i, time_td, mean_td, remaining_td))
            
    print('\nProcessing batches complete. {} apps written'.format(apps_written))

In [86]:
def reset_index(download_path, index_filename):
    """Reset index in file to 0."""
    rel_path = os.path.join(download_path, index_filename)
    
    with open(rel_path, 'w') as f:
        print(0, file=f)
        

def get_index(download_path, index_filename):
    """Retrieve index from file, returning 0 if file not found."""
    try:
        rel_path = os.path.join(download_path, index_filename)

        with open(rel_path, 'r') as f:
            index = int(f.readline())
    
    except FileNotFoundError:
        index = 0
        
    return index


def prepare_data_file(download_path, filename, index, columns):
    """Create file and write headers if index is 0."""
    if index == 0:
        rel_path = os.path.join(download_path, filename)

        with open(rel_path, 'w', newline='') as f:
            writer = csv.DictWriter(f, fieldnames=columns)
            writer.writeheader()

In [87]:
def parse_steam_request(appid, name):
    """Unique parser to handle data from Steam Store API.
    
    Returns : json formatted data (dict-like)
    """
    url = "http://store.steampowered.com/api/appdetails/"
    parameters = {"appids": appid}
    
    json_data = get_request(url, parameters=parameters)
    json_app_data = json_data[str(appid)]
    
    if json_app_data['success']:
        data = json_app_data['data']
    else:
        data = {'name': name, 'steam_appid': appid}
        
    return data


# Set file parameters
#download_path = '../data/download'
download_path = ''
steam_app_data = 'steam_app_data.csv'
steam_index = 'steam_index.txt'

steam_columns = [
    'type', 'name', 'steam_appid', 'required_age', 'is_free', 'controller_support',
    'dlc', 'detailed_description', 'about_the_game', 'short_description', 'fullgame',
    'supported_languages', 'header_image', 'website', 'pc_requirements', 'mac_requirements',
    'linux_requirements', 'legal_notice', 'drm_notice', 'ext_user_account_notice',
    'developers', 'publishers', 'demos', 'price_overview', 'packages', 'package_groups',
    'platforms', 'metacritic', 'reviews', 'categories', 'genres', 'screenshots',
    'movies', 'recommendations', 'achievements', 'release_date', 'support_info',
    'background', 'content_descriptors'
]

# Overwrites last index for demonstration (would usually store highest index so can continue across sessions)
reset_index(download_path, steam_index)

# Retrieve last index downloaded from file
index = get_index(download_path, steam_index)

# Wipe or create data file and write headers if index is 0
prepare_data_file(download_path, steam_app_data, index, steam_columns)

# Set end and chunksize for demonstration - remove to run through entire app list
process_batches(
    parser=parse_steam_request,
    app_list=app_list,
    download_path=download_path,
    data_filename=steam_app_data,
    index_filename=steam_index,
    columns=steam_columns,
    begin=index,
    end=10,
    batchsize=5
)

Starting at index 0:

Exported lines 0-4 to steam_app_data.csv. Batch 0 time: 0:00:08 (avg: 0:00:08, remaining: 0:00:08)
Exported lines 5-9 to steam_app_data.csv. Batch 1 time: 0:00:08 (avg: 0:00:08, remaining: 0:00:00)

Processing batches complete. 10 apps written


In [88]:
# read from stored csv
pd.read_csv('steam_app_data.csv').info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 39 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   type                     10 non-null     object 
 1   name                     10 non-null     object 
 2   steam_appid              10 non-null     int64  
 3   required_age             10 non-null     int64  
 4   is_free                  10 non-null     bool   
 5   controller_support       0 non-null      float64
 6   dlc                      1 non-null      object 
 7   detailed_description     10 non-null     object 
 8   about_the_game           10 non-null     object 
 9   short_description        10 non-null     object 
 10  fullgame                 0 non-null      float64
 11  supported_languages      10 non-null     object 
 12  header_image             10 non-null     object 
 13  website                  2 non-null      object 
 14  pc_requirements          10 n

In [89]:
def parse_steamspy_request(appid, name):
    """Parser to handle SteamSpy API data."""
    url = "https://steamspy.com/api.php"
    parameters = {"request": "appdetails", "appid": appid}
    
    json_data = get_request(url, parameters)
    return json_data


# set files and columns
download_path = ''
steamspy_data = 'steamspy_data.csv'
steamspy_index = 'steamspy_index.txt'

steamspy_columns = [
    'appid', 'name', 'developer', 'publisher', 'score_rank', 'positive',
    'negative', 'userscore', 'owners', 'average_forever', 'average_2weeks',
    'median_forever', 'median_2weeks', 'price', 'initialprice', 'discount',
    'languages', 'genre', 'ccu', 'tags'
]

reset_index(download_path, steamspy_index)
index = get_index(download_path, steamspy_index)

# Wipe data file if index is 0
prepare_data_file(download_path, steamspy_data, index, steamspy_columns)

process_batches(
    parser=parse_steamspy_request,
    app_list=app_list,
    download_path=download_path, 
    data_filename=steamspy_data,
    index_filename=steamspy_index,
    columns=steamspy_columns,
    begin=index,
    end=20,
    batchsize=5,
    pause=0.3
)

Starting at index 0:

Exported lines 0-4 to steamspy_data.csv. Batch 0 time: 0:00:05 (avg: 0:00:05, remaining: 0:00:14)
Exported lines 5-9 to steamspy_data.csv. Batch 1 time: 0:00:05 (avg: 0:00:05, remaining: 0:00:10)
Exported lines 10-14 to steamspy_data.csv. Batch 2 time: 0:00:05 (avg: 0:00:05, remaining: 0:00:05)
Exported lines 15-19 to steamspy_data.csv. Batch 3 time: 0:00:05 (avg: 0:00:05, remaining: 0:00:00)

Processing batches complete. 20 apps written


In [90]:
# inspect downloaded steamspy data
pd.read_csv('steamspy_data.csv').head()

,appid,name,developer,publisher,score_rank,positive,negative,userscore,owners,average_forever,average_2weeks,median_forever,median_2weeks,price,initialprice,discount,languages,genre,ccu,tags
0,10,Counter-Strike,Valve,Valve,NaN,195401,5021,0,"10,000,000 .. 20,000,000",11137,323,190,355,999,999,0,"English, French, German, Italian, Spanish - Sp...",Action,14091,"{'Action': 5387, 'FPS': 4811, 'Multiplayer': 3..."
1,20,Team Fortress Classic,Valve,Valve,NaN,5528,910,0,"5,000,000 .. 10,000,000",223,0,13,0,499,499,0,"English, French, German, Italian, Spanish - Sp...",Action,64,"{'Action': 748, 'FPS': 308, 'Multiplayer': 259..."
2,30,Day of Defeat,Valve,Valve,NaN,5081,556,0,"5,000,000 .. 10,000,000",256,0,26,0,499,499,0,"English, French, German, Italian, Spanish - Spain",Action,150,"{'FPS': 789, 'World War II': 250, 'Multiplayer..."
3,40,Deathmatch Classic,Valve,Valve,NaN,1882,414,0,"5,000,000 .. 10,000,000",96,0,6,0,499,499,0,"English, French, German, Italian, Spanish - Sp...",Action,4,"{'Action': 630, 'FPS': 140, 'Classic': 108, 'M..."
4,50,Half-Life: Opposing Force,Gearbox Software,Valve,NaN,13712,676,0,"2,000,000 .. 5,000,000",1117,0,217,0,499,499,0,"English, French, German, Korean",Action,84,"{'FPS': 884, 'Action': 325, 'Classic': 253, 'S..."
